# Comparaison des différents optimiseurs.

<br/>

##### Arnaud Deleruyelle : Doctorant SIGMA - BioComputing
##### Encadrement: John Klein (SIGMA) - Cristian Versari (BioComputing)

## Partie 1: Initialisation de Tensorflow

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from math import sin
from IPython.core.display import Image, display
from keras.datasets import mnist
import keras

def weight_variable(shape):
    initial = tf.truncated_normal([int(shape[0]),shape[1]], stddev=0.1)
    return tf.Variable(initial, dtype = tf.float32)
 
# The constant value 0.1
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, dtype = tf.float32)

def MLP(x_b,tab_neurones, tab_activation, dropout = 1, name="MLP"):
    with tf.name_scope(name):
        outs = []
        for i in range(len(tab_neurones)):
            if i == 0 :
                input_nb_neurone = x_b.shape[1]
                input_data = x_b
            else:
                input_nb_neurone = tab_neurones[i-1]
                input_data = outs[i-1]
            W = weight_variable([input_nb_neurone, tab_neurones[i]])
            b = bias_variable([tab_neurones[i]])
            if (tab_activation[i] == "relu"):
                out = tf.nn.relu(tf.matmul(input_data, W) + b)
            elif (tab_activation[i] == "sigmoid"):
                out = tf.nn.sigmoid(tf.matmul(input_data, W) + b)
            elif (tab_activation[i] == "tanh"):
                out = tf.tanh(tf.matmul(input_data, W) + b)
            elif (tab_activation[i] == "softmax"):
                out = tf.nn.softmax(tf.matmul(input_data, W) + b)
            else:
                out = tf.matmul(input_data, W) + b
            # gestion du dropout
            if (dropout != 1) & (i != len(tab_neurones)-1):
                out_drop = tf.nn.dropout(out, dropout)
                outs.append(out_drop)
            else:
                outs.append(out)
        return outs



def Fit(x_train,x_test,y_train,y_test,model, verbose = 100):
    sess = tf.InteractiveSession()
    init_vars = tf.variables_initializer(tf.global_variables())
    sess.run(init_vars)
    visualisation = []
    for epoch in range(epochs):
        indices = np.random.choice(x_train.shape[0], size=batch_size)
        x_b = x_train[indices]
        y_b = y_train[indices]
        feed_dict = {x_batch: x_b, y_batch: y_b}
        sess.run(model, feed_dict=feed_dict)
        if epoch%verbose ==0:
            feed_dict_train = {x_batch: x_train, y_batch: y_train}
            feed_dict_test = {x_batch: x_test, y_batch: y_test}
            loss_train = sess.run(avg_llh, feed_dict=feed_dict_train)
            loss_test = sess.run(avg_llh, feed_dict=feed_dict_test)
            print("Epoch: ", epoch)
            print("Loss train = ", loss_train)
            print("Loss test = ", loss_test)
            visualisation.append(np.array([epoch,loss_train,loss_test]))
    return (sess,np.array(visualisation))

    
def ES(x_train,x_test,y_train,y_test,model,n = 100, p = 10, verbose = 500):
    sess = tf.Session()
    init_vars = tf.variables_initializer(tf.global_variables())
    sess.run(init_vars)
    saver = tf.train.Saver()
    visualisation = []
    j = 0
    step = 0
    best_loss = np.inf
    while j < p:
        # train n steps
        for i in range(n):
            indices = np.random.choice(x_train.shape[0], size=batch_size)
            x_b = x_train[indices]
            y_b = y_train[indices]
            feed_dict = {x_batch: x_b, y_batch: y_b}
            sess.run(model, feed_dict=feed_dict)
            step = step+1
        # compute en print actual score
        feed_dict_train = {x_batch: x_train, y_batch: y_train}
        feed_dict_test = {x_batch: x_test, y_batch: y_test}
        loss_train = sess.run(avg_llh, feed_dict=feed_dict_train)
        loss_test = sess.run(avg_llh, feed_dict=feed_dict_test)
        print("Loss train = ", loss_train)
        print("Loss test = ", loss_test)
        visualisation.append(np.array([step,loss_train,loss_test]))
        #compare loss
        if loss_test < best_loss:
            save_path = saver.save(sess, "/tmp/best_model.ckpt")
            j = 0
            best_loss = loss_test
        else:
            j = j+1
    best_model = saver.restore(sess, "/tmp/best_model.ckpt")
    return (sess,np.array(visualisation))

def Predict(x_test,session):
    feed_dict = {x_batch: x_test}
    yh_vals = session.run((y_pred), feed_dict=feed_dict)
    return yh_vals

Using TensorFlow backend.


In [2]:
# vars
x_batch = tf.placeholder(shape=[None, 784], dtype=tf.float32, name='x_batch')
y_batch = tf.placeholder(shape=[None, 10], dtype=tf.float32, name='y_batch')
batch_size = 1000
epochs = 2400

# data
num_classes = 10

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Partie 2: Comparaison des meilleurs résultats.

In [10]:
Mom = tf.train.MomentumOptimizer(learning_rate=0.01,momentum = 0.99)
RMS = tf.train.RMSPropOptimizer(learning_rate=0.001)
Adam = tf.train.AdamOptimizer(learning_rate=0.001)

In [4]:
mlp_mnist_high = MLP(x_batch,[512,512,10], ['sigmoid','sigmoid','identity'])

mlp_mnist_low = MLP(x_batch,[16,16,10], ['sigmoid','sigmoid','identity'])



In [5]:
# Adam case

# output layer
y_pred = mlp_mnist_high[-1]

# loss function
loglikelihoods = tf.nn.softmax_cross_entropy_with_logits_v2(logits = y_pred, labels = y_batch)
avg_llh = tf.reduce_mean(loglikelihoods)

model = Adam.minimize(avg_llh)

# mlp_hight_mnist

accuracys = []

for i in range(5):
    fit,visu = ES(x_train,x_test,y_train,y_test,model)
    result_mnist = Predict(x_test,fit)
    correct_prediction = np.equal(np.argmax(result_mnist, 1), np.argmax(y_test, 1))
    accuracy = np.mean(correct_prediction.astype(float))
    accuracys.append(accuracy)
    print(accuracy)

accuracy = np.mean(accuracys, axis = 0)
print("final accuracy Adam high: " + str(accuracy))

Loss train =  0.34341732
Loss test =  0.3337241
Loss train =  0.26220304
Loss test =  0.2595427
Loss train =  0.22343595
Loss test =  0.22410175
Loss train =  0.19390236
Loss test =  0.19924612
Loss train =  0.16643207
Loss test =  0.17353375
Loss train =  0.14353716
Loss test =  0.15564854
Loss train =  0.12461168
Loss test =  0.13882454
Loss train =  0.11105855
Loss test =  0.12943183
Loss train =  0.09467477
Loss test =  0.114418015
Loss train =  0.08339808
Loss test =  0.10729346
Loss train =  0.074605174
Loss test =  0.100698866
Loss train =  0.06394834
Loss test =  0.09421771
Loss train =  0.05687427
Loss test =  0.08884849
Loss train =  0.049596418
Loss test =  0.08418958
Loss train =  0.044596836
Loss test =  0.07924598
Loss train =  0.037404396
Loss test =  0.075825796
Loss train =  0.03373613
Loss test =  0.07716163
Loss train =  0.029510254
Loss test =  0.07454587
Loss train =  0.026007243
Loss test =  0.072447546
Loss train =  0.022487182
Loss test =  0.07075006
Loss train 

Loss train =  0.1630489
Loss test =  0.1733188
Loss train =  0.14128476
Loss test =  0.15519704
Loss train =  0.124098666
Loss test =  0.13715169
Loss train =  0.10637804
Loss test =  0.12408887
Loss train =  0.09390031
Loss test =  0.113868505
Loss train =  0.08134271
Loss test =  0.10467224
Loss train =  0.07192508
Loss test =  0.09736756
Loss train =  0.06487639
Loss test =  0.090776026
Loss train =  0.056080405
Loss test =  0.0877971
Loss train =  0.048704557
Loss test =  0.0819574
Loss train =  0.04381801
Loss test =  0.08048361
Loss train =  0.036453377
Loss test =  0.0751509
Loss train =  0.03230619
Loss test =  0.071725465
Loss train =  0.028283834
Loss test =  0.07164492
Loss train =  0.024452712
Loss test =  0.06757876
Loss train =  0.02150164
Loss test =  0.0676858
Loss train =  0.01845065
Loss test =  0.067980155
Loss train =  0.016095072
Loss test =  0.06613305
Loss train =  0.014606606
Loss test =  0.06633941
Loss train =  0.011794758
Loss test =  0.06444624
Loss train = 

In [6]:
y_pred = mlp_mnist_low[-1]

# loss function
loglikelihoods = tf.nn.softmax_cross_entropy_with_logits_v2(logits = y_pred, labels = y_batch)
avg_llh = tf.reduce_mean(loglikelihoods)

model = Adam.minimize(avg_llh)

accuracys = []

for i in range(5):
    fit,visu = ES(x_train,x_test,y_train,y_test,model)
    result_mnist = Predict(x_test,fit)
    correct_prediction = np.equal(np.argmax(result_mnist, 1), np.argmax(y_test, 1))
    accuracy = np.mean(correct_prediction.astype(float))
    accuracys.append(accuracy)
    print(accuracy)

accuracy = np.mean(accuracys, axis = 0)
print("final accuracy Adam low: " + str(accuracy))

Loss train =  2.1971745
Loss test =  2.196082
Loss train =  1.8392568
Loss test =  1.8382012
Loss train =  1.5415107
Loss test =  1.5405941
Loss train =  1.3426973
Loss test =  1.3406137
Loss train =  1.1880274
Loss test =  1.1859969
Loss train =  1.0690932
Loss test =  1.0685809
Loss train =  0.9763003
Loss test =  0.97756463
Loss train =  0.88672423
Loss test =  0.88944006
Loss train =  0.7901844
Loss test =  0.7934645
Loss train =  0.6915395
Loss test =  0.69562465
Loss train =  0.60708046
Loss test =  0.610386
Loss train =  0.5381425
Loss test =  0.54236895
Loss train =  0.48108152
Loss test =  0.48571846
Loss train =  0.43510652
Loss test =  0.4410135
Loss train =  0.39846158
Loss test =  0.40732062
Loss train =  0.36832413
Loss test =  0.37904492
Loss train =  0.34436437
Loss test =  0.35739553
Loss train =  0.32314798
Loss test =  0.33760792
Loss train =  0.3064132
Loss test =  0.32191056
Loss train =  0.2914349
Loss test =  0.30782822
Loss train =  0.2782228
Loss test =  0.2970

Loss train =  0.110522166
Loss test =  0.18215132
Loss train =  0.109436534
Loss test =  0.18119647
Loss train =  0.1082599
Loss test =  0.17984127
Loss train =  0.108116694
Loss test =  0.18160176
Loss train =  0.10700951
Loss test =  0.18217146
Loss train =  0.106002994
Loss test =  0.18117663
Loss train =  0.10563118
Loss test =  0.1820822
Loss train =  0.1045658
Loss test =  0.18090682
Loss train =  0.10397993
Loss test =  0.18178602
Loss train =  0.10368716
Loss test =  0.1824835
Loss train =  0.103313
Loss test =  0.18291228
Loss train =  0.101838544
Loss test =  0.18086997
Loss train =  0.101351045
Loss test =  0.18062176
INFO:tensorflow:Restoring parameters from /tmp/best_model.ckpt
0.9478
Loss train =  2.1694431
Loss test =  2.1677449
Loss train =  1.8057716
Loss test =  1.8056766
Loss train =  1.5364382
Loss test =  1.539608
Loss train =  1.3357211
Loss test =  1.339751
Loss train =  1.1572229
Loss test =  1.1598864
Loss train =  1.0136539
Loss test =  1.0153266
Loss train = 

Loss train =  0.15793015
Loss test =  0.20718744
Loss train =  0.15608887
Loss test =  0.20586683
Loss train =  0.15400124
Loss test =  0.20540307
Loss train =  0.15239556
Loss test =  0.20515573
Loss train =  0.15100421
Loss test =  0.20449653
Loss train =  0.1497826
Loss test =  0.20203687
Loss train =  0.14818542
Loss test =  0.20080788
Loss train =  0.14658518
Loss test =  0.20001397
Loss train =  0.14482227
Loss test =  0.20115946
Loss train =  0.1433643
Loss test =  0.20186794
Loss train =  0.14230737
Loss test =  0.20036392
Loss train =  0.14089805
Loss test =  0.19938333
Loss train =  0.13953055
Loss test =  0.19874014
Loss train =  0.13831095
Loss test =  0.19831276
Loss train =  0.13642776
Loss test =  0.19801465
Loss train =  0.13562185
Loss test =  0.19798388
Loss train =  0.13428238
Loss test =  0.19596533
Loss train =  0.13287897
Loss test =  0.19674167
Loss train =  0.13165389
Loss test =  0.19670783
Loss train =  0.13089424
Loss test =  0.19668144
Loss train =  0.129396

In [11]:
y_pred = mlp_mnist_high[-1]

# loss function
loglikelihoods = tf.nn.softmax_cross_entropy_with_logits_v2(logits = y_pred, labels = y_batch)
avg_llh = tf.reduce_mean(loglikelihoods)

model = Mom.minimize(avg_llh)

accuracys = []

for i in range(5):
    fit,visu = ES(x_train,x_test,y_train,y_test,model)
    result_mnist = Predict(x_test,fit)
    correct_prediction = np.equal(np.argmax(result_mnist, 1), np.argmax(y_test, 1))
    accuracy = np.mean(correct_prediction.astype(float))
    accuracys.append(accuracy)
    print(accuracy)

accuracy = np.mean(accuracys, axis = 0)
print("final accuracy SGD Momentum high: " + str(accuracy))

Loss train =  0.6107767
Loss test =  0.5987829
Loss train =  0.42031616
Loss test =  0.40027383
Loss train =  0.3236577
Loss test =  0.31311867
Loss train =  0.2951426
Loss test =  0.28683007
Loss train =  0.26088777
Loss test =  0.2549882
Loss train =  0.23942217
Loss test =  0.23630047
Loss train =  0.22090335
Loss test =  0.22115071
Loss train =  0.21001804
Loss test =  0.21220268
Loss train =  0.194551
Loss test =  0.19482026
Loss train =  0.1874166
Loss test =  0.1898051
Loss train =  0.17842396
Loss test =  0.18187389
Loss train =  0.16406651
Loss test =  0.16919455
Loss train =  0.15432373
Loss test =  0.16043308
Loss train =  0.14699501
Loss test =  0.15545212
Loss train =  0.1382858
Loss test =  0.14814168
Loss train =  0.13299803
Loss test =  0.14276196
Loss train =  0.12557958
Loss test =  0.13654837
Loss train =  0.11927469
Loss test =  0.13181461
Loss train =  0.11512691
Loss test =  0.1284624
Loss train =  0.112273715
Loss test =  0.12629558
Loss train =  0.10461293
Loss 

Loss train =  0.013011976
Loss test =  0.066593066
Loss train =  0.012154156
Loss test =  0.06556865
Loss train =  0.012278116
Loss test =  0.066517785
INFO:tensorflow:Restoring parameters from /tmp/best_model.ckpt
0.9798
Loss train =  0.5950978
Loss test =  0.5726916
Loss train =  0.41910878
Loss test =  0.39909664
Loss train =  0.33634037
Loss test =  0.3234419
Loss train =  0.28652996
Loss test =  0.27145287
Loss train =  0.2575142
Loss test =  0.24964783
Loss train =  0.23645094
Loss test =  0.23228821
Loss train =  0.22063807
Loss test =  0.21733421
Loss train =  0.20505661
Loss test =  0.20606464
Loss train =  0.19096555
Loss test =  0.19189662
Loss train =  0.17987597
Loss test =  0.18146181
Loss train =  0.17268197
Loss test =  0.17614785
Loss train =  0.16093412
Loss test =  0.16482392
Loss train =  0.15517549
Loss test =  0.16075043
Loss train =  0.14712878
Loss test =  0.15336552
Loss train =  0.13723788
Loss test =  0.14277865
Loss train =  0.13202909
Loss test =  0.1401373

Loss train =  0.016807439
Loss test =  0.0666666
Loss train =  0.016071344
Loss test =  0.06585499
Loss train =  0.015931625
Loss test =  0.06691323
Loss train =  0.015921796
Loss test =  0.06688739
Loss train =  0.014790925
Loss test =  0.06697236
Loss train =  0.014850535
Loss test =  0.066831455
Loss train =  0.01422185
Loss test =  0.06737535
Loss train =  0.013907149
Loss test =  0.067903474
Loss train =  0.014007746
Loss test =  0.06726028
Loss train =  0.013193349
Loss test =  0.06629142
INFO:tensorflow:Restoring parameters from /tmp/best_model.ckpt
0.9795
Loss train =  0.5876543
Loss test =  0.5731119
Loss train =  0.41284674
Loss test =  0.39200848
Loss train =  0.33559173
Loss test =  0.325904
Loss train =  0.29232043
Loss test =  0.2829631
Loss train =  0.2662199
Loss test =  0.25932494
Loss train =  0.23995154
Loss test =  0.2336318
Loss train =  0.22645195
Loss test =  0.22306402
Loss train =  0.21165267
Loss test =  0.2091626
Loss train =  0.19866228
Loss test =  0.197080

In [12]:
y_pred = mlp_mnist_low[-1]

# loss function
loglikelihoods = tf.nn.softmax_cross_entropy_with_logits_v2(logits = y_pred, labels = y_batch)
avg_llh = tf.reduce_mean(loglikelihoods)

model = Mom.minimize(avg_llh)

accuracys = []

for i in range(5):
    fit,visu = ES(x_train,x_test,y_train,y_test,model)
    result_mnist = Predict(x_test,fit)
    correct_prediction = np.equal(np.argmax(result_mnist, 1), np.argmax(y_test, 1))
    accuracy = np.mean(correct_prediction.astype(float))
    accuracys.append(accuracy)
    print(accuracy)

accuracy = np.mean(accuracys, axis = 0)
print("final accuracy SGD Momentum low: " + str(accuracy))

Loss train =  2.296639
Loss test =  2.2968023
Loss train =  2.240473
Loss test =  2.2397056
Loss train =  1.8888515
Loss test =  1.8805504
Loss train =  1.3600245
Loss test =  1.3510551
Loss train =  1.0256981
Loss test =  1.0192798
Loss train =  0.7869724
Loss test =  0.7753007
Loss train =  0.6227908
Loss test =  0.6058772
Loss train =  0.50909215
Loss test =  0.49208066
Loss train =  0.4409659
Loss test =  0.429289
Loss train =  0.3945285
Loss test =  0.38874298
Loss train =  0.35957825
Loss test =  0.3593933
Loss train =  0.33176443
Loss test =  0.33543384
Loss train =  0.30455986
Loss test =  0.31275922
Loss train =  0.28572664
Loss test =  0.296564
Loss train =  0.26705417
Loss test =  0.2841797
Loss train =  0.25175846
Loss test =  0.2689535
Loss train =  0.2390108
Loss test =  0.2594115
Loss train =  0.22757334
Loss test =  0.24950403
Loss train =  0.21773209
Loss test =  0.2418625
Loss train =  0.2100154
Loss test =  0.23744304
Loss train =  0.20212503
Loss test =  0.22996414


Loss train =  0.10032629
Loss test =  0.16388962
Loss train =  0.09948767
Loss test =  0.16113095
Loss train =  0.09916499
Loss test =  0.16482982
Loss train =  0.098110504
Loss test =  0.16293962
Loss train =  0.09805844
Loss test =  0.1631269
Loss train =  0.09813378
Loss test =  0.16493358
Loss train =  0.09583437
Loss test =  0.16141057
Loss train =  0.09664159
Loss test =  0.1617868
Loss train =  0.0958856
Loss test =  0.16420308
Loss train =  0.09613616
Loss test =  0.16121022
Loss train =  0.0956786
Loss test =  0.1643554
Loss train =  0.094678044
Loss test =  0.1641305
INFO:tensorflow:Restoring parameters from /tmp/best_model.ckpt
0.9512
Loss train =  2.2973528
Loss test =  2.297533
Loss train =  2.2455647
Loss test =  2.2436903
Loss train =  1.7184359
Loss test =  1.7099197
Loss train =  1.2442
Loss test =  1.241247
Loss train =  0.96192396
Loss test =  0.9655338
Loss train =  0.7637423
Loss test =  0.75965977
Loss train =  0.6300656
Loss test =  0.62178105
Loss train =  0.522

Loss train =  0.3467382
Loss test =  0.34683856
Loss train =  0.31455803
Loss test =  0.3187374
Loss train =  0.2891837
Loss test =  0.29220343
Loss train =  0.26812303
Loss test =  0.27488902
Loss train =  0.25325257
Loss test =  0.26215473
Loss train =  0.24162462
Loss test =  0.25317958
Loss train =  0.2288668
Loss test =  0.24098228
Loss train =  0.2198331
Loss test =  0.23481406
Loss train =  0.2115263
Loss test =  0.22723706
Loss train =  0.20434764
Loss test =  0.224402
Loss train =  0.19722822
Loss test =  0.2165997
Loss train =  0.19115469
Loss test =  0.21138605
Loss train =  0.1863167
Loss test =  0.20862952
Loss train =  0.18234983
Loss test =  0.20548916
Loss train =  0.17642175
Loss test =  0.20004675
Loss train =  0.17243665
Loss test =  0.19779016
Loss train =  0.16880189
Loss test =  0.19489332
Loss train =  0.16546546
Loss test =  0.19156557
Loss train =  0.16287145
Loss test =  0.19274068
Loss train =  0.15948197
Loss test =  0.18813565
Loss train =  0.15538198
Loss 

In [13]:
y_pred = mlp_mnist_high[-1]

# loss function
loglikelihoods = tf.nn.softmax_cross_entropy_with_logits_v2(logits = y_pred, labels = y_batch)
avg_llh = tf.reduce_mean(loglikelihoods)

model = RMS.minimize(avg_llh)

accuracys = []

for i in range(5):
    fit,visu = ES(x_train,x_test,y_train,y_test,model)
    result_mnist = Predict(x_test,fit)
    correct_prediction = np.equal(np.argmax(result_mnist, 1), np.argmax(y_test, 1))
    accuracy = np.mean(correct_prediction.astype(float))
    accuracys.append(accuracy)
    print(accuracy)

accuracy = np.mean(accuracys, axis = 0)
print("final accuracy RMSProp high: " + str(accuracy))

Loss train =  1.9945116
Loss test =  1.9855254
Loss train =  0.47322428
Loss test =  0.45588973
Loss train =  0.31920934
Loss test =  0.31065553
Loss train =  0.25105608
Loss test =  0.25093138
Loss train =  0.21569705
Loss test =  0.22285932
Loss train =  0.17968757
Loss test =  0.18303303
Loss train =  0.14325379
Loss test =  0.15076485
Loss train =  0.14337547
Loss test =  0.15250579
Loss train =  0.106344916
Loss test =  0.12102378
Loss train =  0.098016396
Loss test =  0.11609243
Loss train =  0.076145746
Loss test =  0.095934
Loss train =  0.074543394
Loss test =  0.09864872
Loss train =  0.0659006
Loss test =  0.09466406
Loss train =  0.055902176
Loss test =  0.08621102
Loss train =  0.055825397
Loss test =  0.08879602
Loss train =  0.04263851
Loss test =  0.08024815
Loss train =  0.03358606
Loss test =  0.07276241
Loss train =  0.028747616
Loss test =  0.06771604
Loss train =  0.023534544
Loss test =  0.069071725
Loss train =  0.02011297
Loss test =  0.06615237
Loss train =  0.

Loss train =  0.023804637
Loss test =  0.0766782
Loss train =  0.013991739
Loss test =  0.06828529
Loss train =  0.020032478
Loss test =  0.0782163
Loss train =  0.018632697
Loss test =  0.08005939
Loss train =  0.01608471
Loss test =  0.07873743
Loss train =  0.010906396
Loss test =  0.078599535
Loss train =  0.0057604047
Loss test =  0.0686059
Loss train =  0.0059698625
Loss test =  0.07376417
Loss train =  0.008629822
Loss test =  0.08238258
Loss train =  0.003479426
Loss test =  0.074200034
INFO:tensorflow:Restoring parameters from /tmp/best_model.ckpt
0.9816
final accuracy RMSProp high: 0.98178


In [14]:
y_pred = mlp_mnist_low[-1]

# loss function
loglikelihoods = tf.nn.softmax_cross_entropy_with_logits_v2(logits = y_pred, labels = y_batch)
avg_llh = tf.reduce_mean(loglikelihoods)

model = RMS.minimize(avg_llh)

accuracys = []

for i in range(5):
    fit,visu = ES(x_train,x_test,y_train,y_test,model)
    result_mnist = Predict(x_test,fit)
    correct_prediction = np.equal(np.argmax(result_mnist, 1), np.argmax(y_test, 1))
    accuracy = np.mean(correct_prediction.astype(float))
    accuracys.append(accuracy)
    print(accuracy)

accuracy = np.mean(accuracys, axis = 0)
print("final accuracy RMSProp low: " + str(accuracy))

Loss train =  2.3014088
Loss test =  2.3014228
Loss train =  2.2561562
Loss test =  2.2549837
Loss train =  2.026391
Loss test =  2.0236864
Loss train =  1.7361197
Loss test =  1.7331342
Loss train =  1.4759924
Loss test =  1.472206
Loss train =  1.2708448
Loss test =  1.2660682
Loss train =  1.1151125
Loss test =  1.1096395
Loss train =  0.9906667
Loss test =  0.98437595
Loss train =  0.8795749
Loss test =  0.872586
Loss train =  0.7809833
Loss test =  0.7729001
Loss train =  0.69921976
Loss test =  0.6919641
Loss train =  0.6325383
Loss test =  0.62505245
Loss train =  0.575562
Loss test =  0.56785965
Loss train =  0.52543926
Loss test =  0.51847607
Loss train =  0.48329428
Loss test =  0.47787333
Loss train =  0.4452722
Loss test =  0.44109726
Loss train =  0.41274688
Loss test =  0.41108346
Loss train =  0.38442695
Loss test =  0.38382587
Loss train =  0.36026508
Loss test =  0.36182168
Loss train =  0.3379674
Loss test =  0.34177786
Loss train =  0.32032293
Loss test =  0.32654923

Loss train =  0.13833575
Loss test =  0.19106026
Loss train =  0.13818994
Loss test =  0.19173421
Loss train =  0.1362686
Loss test =  0.18928619
Loss train =  0.1345166
Loss test =  0.18917039
Loss train =  0.13327467
Loss test =  0.1878457
Loss train =  0.13335745
Loss test =  0.18813227
Loss train =  0.1310409
Loss test =  0.18694457
Loss train =  0.12985271
Loss test =  0.18577576
Loss train =  0.12948175
Loss test =  0.18742749
Loss train =  0.12807228
Loss test =  0.18508175
Loss train =  0.12711228
Loss test =  0.18642847
Loss train =  0.12560737
Loss test =  0.1853385
Loss train =  0.12475002
Loss test =  0.18459627
Loss train =  0.12511681
Loss test =  0.18565468
Loss train =  0.123097904
Loss test =  0.18554045
Loss train =  0.12231939
Loss test =  0.18490395
Loss train =  0.121299714
Loss test =  0.18544179
Loss train =  0.12031355
Loss test =  0.18236214
Loss train =  0.11946854
Loss test =  0.1847974
Loss train =  0.119161196
Loss test =  0.18545108
Loss train =  0.1179590

Loss train =  0.20111175
Loss test =  0.22196011
Loss train =  0.19624154
Loss test =  0.21743664
Loss train =  0.19099884
Loss test =  0.21451938
Loss train =  0.18662535
Loss test =  0.20955503
Loss train =  0.18260463
Loss test =  0.20646778
Loss train =  0.17967945
Loss test =  0.20362371
Loss train =  0.17518912
Loss test =  0.19960093
Loss train =  0.17216907
Loss test =  0.1983737
Loss train =  0.16897306
Loss test =  0.19506472
Loss train =  0.16614112
Loss test =  0.19396119
Loss train =  0.16325036
Loss test =  0.1913084
Loss train =  0.16017142
Loss test =  0.18925735
Loss train =  0.15782402
Loss test =  0.18660888
Loss train =  0.15595059
Loss test =  0.1865193
Loss train =  0.15343077
Loss test =  0.1847597
Loss train =  0.15121919
Loss test =  0.18314324
Loss train =  0.14974472
Loss test =  0.18263166
Loss train =  0.14759278
Loss test =  0.18188423
Loss train =  0.1460335
Loss test =  0.18089919
Loss train =  0.14425287
Loss test =  0.1782198
Loss train =  0.14225142
L

Loss train =  0.10413234
Loss test =  0.18208197
Loss train =  0.10410661
Loss test =  0.18359539
Loss train =  0.10328958
Loss test =  0.18549801
Loss train =  0.103043914
Loss test =  0.18473262
Loss train =  0.10198152
Loss test =  0.18521905
Loss train =  0.10143875
Loss test =  0.18361965
Loss train =  0.100621544
Loss test =  0.18379942
INFO:tensorflow:Restoring parameters from /tmp/best_model.ckpt
0.9521
final accuracy RMSProp low: 0.95206


## Partie 3: Comparaison en temps.

In [15]:
%%time

# Adam case

y_pred = mlp_mnist_low[-1]

# loss function
loglikelihoods = tf.nn.softmax_cross_entropy_with_logits_v2(logits = y_pred, labels = y_batch)
avg_llh = tf.reduce_mean(loglikelihoods)

model = Adam.minimize(avg_llh)

fit,visu = ES(x_train,x_test,y_train,y_test,model)
result_mnist = Predict(x_test,fit)
correct_prediction = np.equal(np.argmax(result_mnist, 1), np.argmax(y_test, 1))
accuracy = np.mean(correct_prediction.astype(float))
print("accuracy Adam: " + str(accuracy))

Loss train =  2.1879077
Loss test =  2.1880574
Loss train =  1.8488953
Loss test =  1.8497372
Loss train =  1.5182027
Loss test =  1.517862
Loss train =  1.2653458
Loss test =  1.2650241
Loss train =  1.085089
Loss test =  1.0853744
Loss train =  0.94654775
Loss test =  0.94707775
Loss train =  0.830037
Loss test =  0.83075774
Loss train =  0.72115064
Loss test =  0.72043324
Loss train =  0.6257169
Loss test =  0.6248905
Loss train =  0.55344546
Loss test =  0.5530878
Loss train =  0.49683315
Loss test =  0.49661112
Loss train =  0.45104936
Loss test =  0.45052266
Loss train =  0.41420227
Loss test =  0.41532472
Loss train =  0.3830336
Loss test =  0.3853568
Loss train =  0.35864857
Loss test =  0.36162478
Loss train =  0.33748293
Loss test =  0.34204
Loss train =  0.32042715
Loss test =  0.32671654
Loss train =  0.30552924
Loss test =  0.31324083
Loss train =  0.29210258
Loss test =  0.3019124
Loss train =  0.28048798
Loss test =  0.2897919
Loss train =  0.2700301
Loss test =  0.28196

In [16]:
%%time

# SGD Momentum case

y_pred = mlp_mnist_low[-1]

# loss function
loglikelihoods = tf.nn.softmax_cross_entropy_with_logits_v2(logits = y_pred, labels = y_batch)
avg_llh = tf.reduce_mean(loglikelihoods)

model = Mom.minimize(avg_llh)

fit,visu = ES(x_train,x_test,y_train,y_test,model)
result_mnist = Predict(x_test,fit)
correct_prediction = np.equal(np.argmax(result_mnist, 1), np.argmax(y_test, 1))
accuracy = np.mean(correct_prediction.astype(float))
print("accuracy Momentum: " + str(accuracy))

Loss train =  2.3025765
Loss test =  2.3025956
Loss train =  2.2572315
Loss test =  2.2563243
Loss train =  1.7762773
Loss test =  1.7725084
Loss train =  1.147182
Loss test =  1.1517891
Loss train =  0.89983135
Loss test =  0.89751446
Loss train =  0.6857326
Loss test =  0.66991866
Loss train =  0.5460648
Loss test =  0.5318275
Loss train =  0.45738003
Loss test =  0.44805536
Loss train =  0.4008966
Loss test =  0.3994289
Loss train =  0.35905105
Loss test =  0.35920557
Loss train =  0.32944655
Loss test =  0.3325204
Loss train =  0.30338374
Loss test =  0.3093109
Loss train =  0.28315818
Loss test =  0.2945974
Loss train =  0.26712564
Loss test =  0.2800232
Loss train =  0.25187343
Loss test =  0.26748025
Loss train =  0.24177405
Loss test =  0.2605102
Loss train =  0.23061523
Loss test =  0.24945271
Loss train =  0.22138369
Loss test =  0.24196845
Loss train =  0.21281259
Loss test =  0.23315957
Loss train =  0.20671602
Loss test =  0.22587305
Loss train =  0.19932765
Loss test =  0

In [17]:
%%time

# RMSProp case

y_pred = mlp_mnist_low[-1]

# loss function
loglikelihoods = tf.nn.softmax_cross_entropy_with_logits_v2(logits = y_pred, labels = y_batch)
avg_llh = tf.reduce_mean(loglikelihoods)

model = RMS.minimize(avg_llh)

fit,visu = ES(x_train,x_test,y_train,y_test,model)
result_mnist = Predict(x_test,fit)
correct_prediction = np.equal(np.argmax(result_mnist, 1), np.argmax(y_test, 1))
accuracy = np.mean(correct_prediction.astype(float))
print("accuracy RMSProp: " + str(accuracy))

Loss train =  2.301665
Loss test =  2.3017724
Loss train =  2.254087
Loss test =  2.2528448
Loss train =  2.0231082
Loss test =  2.0203917
Loss train =  1.804188
Loss test =  1.7993592
Loss train =  1.6066973
Loss test =  1.6006348
Loss train =  1.4336857
Loss test =  1.4279324
Loss train =  1.2921821
Loss test =  1.2885355
Loss train =  1.1669837
Loss test =  1.1648221
Loss train =  1.0446838
Loss test =  1.0437032
Loss train =  0.9267078
Loss test =  0.926659
Loss train =  0.8188104
Loss test =  0.8201721
Loss train =  0.7261634
Loss test =  0.72854203
Loss train =  0.64699584
Loss test =  0.65008754
Loss train =  0.5790563
Loss test =  0.58315206
Loss train =  0.52001303
Loss test =  0.5230391
Loss train =  0.4701524
Loss test =  0.47450107
Loss train =  0.4279704
Loss test =  0.43376493
Loss train =  0.39422727
Loss test =  0.40115532
Loss train =  0.3670995
Loss test =  0.37508595
Loss train =  0.34411973
Loss test =  0.35471046
Loss train =  0.32517666
Loss test =  0.3389988
Loss